In [1]:
from kubernetes import client, config
import uuid

# 加载认证文件
# config.kube_config.load_kube_config("minikube_config.yml")
config.kube_config.load_kube_config("config.yml")
#获取CoreV1Api、BatchV1Api、AppsV1Api、CustomObjectsApi
core_v1 = client.CoreV1Api()  #用于操作核心v1 API组中的资源，如Pod、Service、Node、Namespace等
batch_v1 = client.BatchV1Api()  #用于操作Job等资源
apps_v1 = client.AppsV1Api()  #用于操作Deployment、DaemonSet、ReplicaSet和StatefulSet
custom_api = client.CustomObjectsApi()  #用于操作CRD

In [11]:
# 创建一个namespace
new_namespace = client.V1Namespace()
new_namespace.metadata = client.V1ObjectMeta(name="brody")
print(new_namespace)
core_v1.create_namespace(body=new_namespace)

{'api_version': None,
 'kind': None,
 'metadata': {'annotations': None,
              'creation_timestamp': None,
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': None,
              'name': 'brody',
              'namespace': None,
              'owner_references': None,
              'resource_version': None,
              'self_link': None,
              'uid': None},
 'spec': None,
 'status': None}


{'api_version': 'v1',
 'kind': 'Namespace',
 'metadata': {'annotations': None,
              'creation_timestamp': datetime.datetime(2023, 4, 20, 8, 0, 43, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': None,
              'name': 'brody',
              'namespace': None,
              'owner_references': None,
              'resource_version': '1997457',
              'self_link': '/api/v1/namespaces/brody',
              'uid': '9f9c9f9b-7687-4056-b5ad-1700a9bd4c29'},
 'spec': {'finalizers': ['kubernetes']},
 'status': {'conditions': None, 'phase': 'Active'}}

In [12]:
# 删除一个namespace
core_v1.delete_namespace("brody")

{'api_version': 'v1',
 'code': None,
 'details': None,
 'kind': 'Namespace',
 'message': None,
 'metadata': {'_continue': None,
              'remaining_item_count': None,
              'resource_version': '1998166',
              'self_link': '/api/v1/namespaces/brody'},
 'reason': None,
 'status': "{'phase': 'Terminating'}"}

In [13]:
# 创建pod
nginx_pod = {
    'apiVersion': 'v1',
    'kind': 'Pod',
    'metadata': {
        'name': 'nginx-{}'.format(uuid.uuid4()),
        'labels': {
            "app": "nginx"
        }
    },
    'spec': {
        'containers': [
            {
                'name': 'my-container',
                'image': 'nginx:1.7.9',
                "ports": [
                    {
                        "containerPort": 80
                    }
                ]
            }
        ],
        #指定调度节点
        # 'nodeSelector': {
        #     'kubernetes.io/hostname': 'bigdata4'
        # }
    }
}
"""
# 标记node
kubectl label nodes <node-name> <label-key>=<label-value>
# 展现node的label
kubectl get nodes --show-labels
# 描述node结构
kubectl describe node "nodename"
"""
core_v1.create_namespaced_pod(body=nginx_pod, namespace="default")

{'api_version': 'v1',
 'kind': 'Pod',
 'metadata': {'annotations': None,
              'creation_timestamp': datetime.datetime(2023, 4, 20, 8, 8, 39, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': {'app': 'nginx'},
              'managed_fields': None,
              'name': 'nginx-c0342ab9-e21d-4ca6-8d57-da5606efcebf',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '1998660',
              'self_link': '/api/v1/namespaces/default/pods/nginx-c0342ab9-e21d-4ca6-8d57-da5606efcebf',
              'uid': '7a058dc8-15b1-4c29-b041-f606c369903e'},
 'spec': {'active_deadline_seconds': None,
          'affinity': None,
          'automount_service_account_token': None,
          'containers': [{'args': None,
                          'comman

In [ ]:
# 用客户端来描述pod
# 配置meta信息
pod_metadata = client.V1ObjectMeta(
    name="my-pod",
    labels={
        "app": "my-app",
        "env": "production",
        "team": "devops"
    },
    annotations={
        "key":"value"
    }
)
# 配置Container信息
container = client.V1Container(name="my-container", image="nginx:latest")
# 配置spec信息
pod_spec = client.V1PodSpec(containers=[container])
# 组合成完整pod信息
pod = client.V1Pod(metadata=pod_metadata, spec=pod_spec)
core_v1.create_namespaced_pod(body=pod, namespace="default")

In [19]:
# 更新pod之replace与patch
"""
core_v1.replace和 core_v1.patch是 Kubernetes Python client 提供的两种方法，用于更新 Kubernetes API 对象。它们之间的主要区别如下：
core_v1.replace 方法是用新的对象替换现有对象，而 core_v1.patch 方法是修改现有对象的一部分。
core_v1.replace 方法需要提供完整的对象规范，而 core_v1.patch 方法只需要提供要更新的字段。
core_v1.replace 方法会在 Kubernetes 中创建一个新的版本（version）的对象，而 core_v1.patch 方法只是在现有对象上进行修改。
"""
# 使用patch
patch = [
    {"op": "replace", "path": "/spec/containers/0/image", "value": "nginx:1.22.1"}
]
# 使用 patch_namespaced_pod 方法更新 Pod
core_v1.patch_namespaced_pod(
    name="nginx-c0342ab9-e21d-4ca6-8d57-da5606efcebf",
    namespace="default",
    body=patch
)

{'api_version': 'v1',
 'kind': 'Pod',
 'metadata': {'annotations': {'cni.projectcalico.org/podIP': '10.244.103.73/32',
                              'cni.projectcalico.org/podIPs': '10.244.103.73/32'},
              'creation_timestamp': datetime.datetime(2023, 4, 20, 8, 8, 39, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': {'app': 'nginx'},
              'managed_fields': None,
              'name': 'nginx-c0342ab9-e21d-4ca6-8d57-da5606efcebf',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '2000392',
              'self_link': '/api/v1/namespaces/default/pods/nginx-c0342ab9-e21d-4ca6-8d57-da5606efcebf',
              'uid': '7a058dc8-15b1-4c29-b041-f606c369903e'},
 'spec': {'active_deadline_seconds': None,
          'affinity': N

In [20]:
# 删除pod
core_v1.delete_namespaced_pod("nginx-c0342ab9-e21d-4ca6-8d57-da5606efcebf", namespace="default")

{'api_version': 'v1',
 'kind': 'Pod',
 'metadata': {'annotations': {'cni.projectcalico.org/podIP': '10.244.103.73/32',
                              'cni.projectcalico.org/podIPs': '10.244.103.73/32'},
              'creation_timestamp': datetime.datetime(2023, 4, 20, 8, 8, 39, tzinfo=tzutc()),
              'deletion_grace_period_seconds': 30,
              'deletion_timestamp': datetime.datetime(2023, 4, 20, 8, 21, 13, tzinfo=tzutc()),
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': {'app': 'nginx'},
              'managed_fields': None,
              'name': 'nginx-c0342ab9-e21d-4ca6-8d57-da5606efcebf',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '2000499',
              'self_link': '/api/v1/namespaces/default/pods/nginx-c0342ab9-e21d-4ca6-8d57-da5606efcebf',
              'uid': '7a058dc8-15b1-4c29-b041-f606c369903e'},
 'spec': {'acti

In [2]:
# 创建deployment
nginx_deploy = {
    'apiVersion': 'apps/v1',
    'kind': 'Deployment',
    'metadata': {
        'name': 'nginx-deployment-python',
    },
    'spec': {
        'selector': {
            "matchLabels": {
                "app": "nginx"
            }
        },
        "replicas": 3,
        "template": {
            "metadata": {
                "labels": {
                    "app": "nginx"
                }
            },
            'spec': {
                'containers': [
                    {
                        'name': 'nginx',
                        'image': 'nginx:1.7.9',
                        "ports": [
                            {
                                "containerPort": 80
                            }
                        ]
                    }
                ]
            }
        }
    }
}
apps_v1.create_namespaced_deployment(namespace="default", body=nginx_deploy)

{'api_version': 'apps/v1',
 'kind': 'Deployment',
 'metadata': {'annotations': None,
              'creation_timestamp': datetime.datetime(2023, 4, 12, 3, 0, 30, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'labels': None,
              'managed_fields': [{'api_version': 'apps/v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:spec': {'f:progressDeadlineSeconds': {},
                                                           'f:replicas': {},
                                                           'f:revisionHistoryLimit': {},
                                                           'f:selector': {},
                                                           'f:strategy': {'f:rollingUpdate': {'.': {},
                                   

In [5]:
# 删除Deploy
apps_v1.delete_namespaced_deployment(namespace="default", name="nginx-deployment-python")

{'api_version': 'v1',
 'code': None,
 'details': {'causes': None,
             'group': 'apps',
             'kind': 'deployments',
             'name': 'nginx-deployment-python',
             'retry_after_seconds': None,
             'uid': '8e32bc9e-6c89-4ede-aa9c-45bdd635a313'},
 'kind': 'Status',
 'message': None,
 'metadata': {'_continue': None,
              'remaining_item_count': None,
              'resource_version': None,
              'self_link': None},
 'reason': None,
 'status': 'Success'}

In [2]:
# 要求先在集群内创建好，crd
# 创建一个CR实例
group = "stable.example.com"
version = "v1"
plural = "crontabs"

# 定义CR对象的元数据
metadata = client.V1ObjectMeta(name="crontab-python")

# 定义CR对象的规格
spec = {
    "cronSpec": "* * * * */5",
    "image": "my-awesome-cron-image"
}

# 创建CR对象
crd_object = {
    "apiVersion": f"{group}/{version}",
    "kind": "CronTab",
    "metadata": metadata,
    "spec": spec
}

# 在"default"命名空间中创建CRD对象
custom_api.create_namespaced_custom_object(
    group=group,
    version=version,
    namespace="default",
    plural=plural,
    body=crd_object
)

{'apiVersion': 'stable.example.com/v1',
 'kind': 'CronTab',
 'metadata': {'creationTimestamp': '2023-04-20T08:34:21Z',
  'generation': 1,
  'name': 'crontab-python',
  'namespace': 'default',
  'resourceVersion': '2002556',
  'selfLink': '/apis/stable.example.com/v1/namespaces/default/crontabs/crontab-python',
  'uid': '5cf46903-d985-435b-9688-6a2890545b63'},
 'spec': {'cronSpec': '* * * * */5', 'image': 'my-awesome-cron-image'}}

In [3]:
# 修改cr
spec = {
    "cronSpec": "* * * * */10",
    "image": "my-bad-cron-image"
}
crd_object = {
    "apiVersion": f"{group}/{version}",
    "kind": "CronTab",
    "metadata": metadata,
    "spec": spec
}
custom_api.patch_namespaced_custom_object(group=group, version=version, namespace="default", plural=plural,
                                          name="crontab-python", body=crd_object)

{'apiVersion': 'stable.example.com/v1',
 'kind': 'CronTab',
 'metadata': {'creationTimestamp': '2023-04-20T08:34:21Z',
  'generation': 2,
  'name': 'crontab-python',
  'namespace': 'default',
  'resourceVersion': '2002602',
  'selfLink': '/apis/stable.example.com/v1/namespaces/default/crontabs/crontab-python',
  'uid': '5cf46903-d985-435b-9688-6a2890545b63'},
 'spec': {'cronSpec': '* * * * */10', 'image': 'my-bad-cron-image'}}

In [4]:
# 删除CR对象
custom_api.delete_namespaced_custom_object(group=group, version=version, namespace="default", plural=plural,
                                           name="crontab-python")

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'crontab-python',
  'group': 'stable.example.com',
  'kind': 'crontabs',
  'uid': '5cf46903-d985-435b-9688-6a2890545b63'}}